In [6]:
from IPython.display import JSON

import altair as alt
# Uncomment/run this line to enable Altair in JupyterLab/nteract:
alt.enable_mime_rendering()

import ipywidgets as widgets
from ipywidgets import fixed, interact

from openfisca_core import decompositions, periods
from openfisca_france import FranceTaxBenefitSystem
from openfisca_matplotlib import graphs

%matplotlib inline

In [7]:
tbs = FranceTaxBenefitSystem()

## Display "Revenu disponible" interactively

In [18]:
def calculate_revenu_disponible(age, salaire_de_base, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
            "salaire_de_base": salaire_de_base, # Annual basis
        },
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    revenu_disponible = simulation.calculate("revenu_disponible", period)
    return revenu_disponible

Try with an input text:

In [19]:
interact(
    calculate_revenu_disponible,
    salaire_de_base=widgets.BoundedFloatText(min=0, max=150000, step=100, value=15000, description="Salaire de base"),
    age=fixed(30),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(BoundedFloatText(value=15000.0, description=u'Salaire de base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Note: press Enter after you type a new value to recompute.

Try with two sliders, for "age" and "salaire_de_base":

In [20]:
interact(
    calculate_revenu_disponible,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    salaire_de_base=widgets.FloatSlider(min=0, max=150000, step=100, value=15000, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), FloatSlider(value=15000.0, continuous_update=False, description=u'salaire_de_base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Note: you can edit the number at the right of the slider. Press Enter after you type a new value to recompute.

This is quite slow to recompute and redraw if we don't use `continuous_update=False`.
This is mainly because we create a new simulation each time, in `calculate_revenu_disponible`. I did not find how to update a variable of an existing simulation, I'm not sure it is possible with OpenFisca.

## Display waterfall

In [22]:
def display_waterfall(age, salaire_de_base, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
            "salaire_de_base": salaire_de_base, # Annual basis
        },
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    graphs.draw_waterfall(simulation)

In [23]:
interact(
    display_waterfall,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    salaire_de_base=widgets.FloatSlider(min=0, max=150000, step=100, value=15000, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), FloatSlider(value=15000.0, continuous_update=False, description=u'salaire_de_base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

## Vary "Salaire" and create a "bareme"

In [24]:
def display_bareme(age, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
        },
        "axes": [
            dict(
                count = 10,
                min = 0,
                max = 50000,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    graphs.draw_bareme(
        simulation = simulation,
        x_axis = "salaire_imposable",
        legend_position = 2,
    )

In [25]:
interact(
    display_bareme,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), Output()), _dom_classes=('widget-interact',))